In [0]:
%run ../Includes/School-Setup

In [0]:
%sql
select * from courses;

In [0]:
stream_df = spark.readStream.table("courses")

In [0]:
display(stream_df)

In [0]:
stream_df.createOrReplaceTempView("courses_streaming_tmp_vw")


In [0]:
%sql
select * from courses_streaming_tmp_vw


In [0]:
%sql
select instructor, count(course_id) as total_courses 
  from courses_streaming_tmp_vw 
group by instructor order by total_courses desc 

In [0]:
%sql
select count(*) as test,category from courses_streaming_tmp_vw group by category order by test

In [0]:
%sql
create or replace temp view instructor_counts_tmp_vw as(
  select instructor, count(course_id) as total_courses 
  from courses_streaming_tmp_vw 
group by instructor order by total_courses desc
)

In [0]:
result_stream_df = spark.table("instructor_counts_tmp_vw")

In [0]:
display(result_stream_df)

In [0]:
(
    result_stream_df.writeStream
    .trigger(availableNow=True)
    .outputMode("complete")
    .option("checkpointLocation", "dbfs/mnt/DEA-Book/checkpoints/instructor_counts")
    .table("instructor_counts")
    .awaitTermination()
)

In [0]:
%sql
select * from instructor_counts;

In [0]:
%sql
insert into courses(
  values("C16", "Generative AI", "Pierre B.", "Computer Science", 25),
        ("C17", "Embedded Systems", "Julia S.", "Computer Science", 30),
        ("C18", "Virtual Reality", "Bernard M.", "Computer Science", 35)

)

In [0]:
%sql
INSERT INTO courses
values ("C19", "Compiler Design", "Sophie B.", "Computer Science", 25),
       ("C20", "Signal Processing", "Sam M.", "Computer Science", 30),
       ("C21", "Operating Systems", "Mark H.", "Computer Science", 35)


In [0]:
import pyspark.sql.functions as f

output_stream_df =(
    stream_df.groupBy("instructor")
    .agg(f.count("course_id").alias("total_courses"))
)

In [0]:
display(output_stream_df)

In [0]:
(
    output_stream_df.writeStream
    .trigger(availableNow=True)
    .outputMode("complete")
    .option("checkpointLocation", 
            "dbfs/mnt/DEA-Book/checkpoints/instructor_counts_py")
    .table("instructor_counts_py")
    .awaitTermination()
    )

In [0]:
%sql
select * from instructor_counts_py

In [0]:
intructor_counts_df = spark.read.table("instructor_counts")
display(intructor_counts_df)

## Incremental Data Ingestion